# EXPLIQUER CE QUE L'ON CHERCHE A PREDIRE, INTERET DE LA PROBLEMATIQUE, ETC

Faire avec et sans le match de 11h pour voir la différence de résultat

In [1]:
import string
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
pd.set_option('display.max_columns', 500)

In [4]:
def perc_print(a, b):
    return round((a-b)/a*100)

In [6]:
PATH = '/home/cesar/Desktop/data/'

## Retraitement des données -> Sauvegarder notre base une fois cette opération faite dans Features engineering

In [9]:
data = pd.read_csv(PATH+'atp_matches_2010.csv')
for annee in range(2000, 2020):
    data = pd.concat([data, pd.read_csv(PATH+'atp_matches_'+str(annee)+'.csv')], axis=0, sort=False)

print('La base contient {} observations et {} attributs.'.format(data.shape[0], data.shape[1]))
data.reset_index(inplace=True, drop=True)
nb_obs = data.shape[0]
data['tourney_date'] = pd.to_datetime(data['tourney_date'], yearfirst=True, format='%Y%m%d')
data['year'] = data['tourney_date'].apply(lambda x: x.year)
data = data[~data['tourney_name'].str.startswith('Davis Cup')].copy()
data.dropna(subset=['minutes'], inplace=True)
data.drop_duplicates(inplace=True)
print('{} observations supprimées ({}%).'.format(nb_obs - data.shape[0], perc_print(nb_obs, data.shape[0])))
def contains_ascii_letter(txt):
    gate = False
    for i in range(len(txt)):
        if txt[i] in string.ascii_letters:
            gate = True
            break
    return gate
nb_obs = data.shape[0]
get_ascii_letter = np.zeros(data.shape[0], dtype=bool)
get_ascii_letter = data['score'].apply(contains_ascii_letter)
data = data[~get_ascii_letter].copy()
print('{} observations supprimées ({}%).'.format(nb_obs - data.shape[0], perc_print(nb_obs, data.shape[0])))
data.loc[data[(data['tourney_id']=='2017-0308')&(data['match_num']==297)].index[0], 'minutes'] = 98
data.loc[data[(data['tourney_id']=='2016-M001')&(data['match_num']==294)].index[0], 'minutes'] = 90
data.reset_index(inplace=True, drop=True)

La base contient 64694 observations et 49 attributs.
11093 observations supprimées (17%).
1678 observations supprimées (3%).


REMARQUE : On ne se base pas sur la table avec player / opponnent

# Modèles de base

Nous allons présenter dans ce notebook une première série de modèles basés sur l'exploitation des attributs contenus dans notre base de données afin d'analyser les résultats obtenus par ce biais avant de complexifier le modèle en procédant à une phase de création de features.  
Ces résultats serviront de baseline par la suite afin de juger de la qualité des modèles qui le suivront. La mesure de qualité du modèle choisi sera la Mean Squared Error (MSE).  

La plupart de nos variables sont des statistiques concernant directement le match et ne sont donc pas utilisables dans le cadre d'une prédiction. Ces variables devront être aggrégés avant d'être utilisable pour caractériser un joueur. 
  
Les différentes opérations effectuées sont :
- Création d'une base d'apprentissage et de validation
- Séparation des attributs et du phénomène à expliquer
- Traitement des variables catégorielles
- Régression linéaire
- Régression Lasso
- Régression polynomiale (déjà une création de features... à voir)
- Arbre de décisions
- Random Forest
- Adaboost

## Création d'une base d'appprentissage et de validation

In [8]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)